In [116]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

train_input = pd.read_csv('train.csv')
train_input

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


Lets start by creating different data from the datetime column. What seems relevant here is month and time the hour of the day, so lets follow that assumption.

In [117]:
import datetime

hour = []
month = []

for date in train_input.datetime:
    converted = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    hour.append(converted.time().hour)
    month.append(converted.date().month)

train_input['hour'] = hour
train_input['month'] = month

train_input = train_input.drop('datetime', axis=1)

Lets check how the dataframe is after

In [118]:
train_input

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,month
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,0,1
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2,1
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,3,1
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,19,12
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,20,12
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,21,12
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,22,12


Lets check high correlation features

In [119]:
train_input.corr()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,month
season,1.000000,0.029368,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.096758,0.164011,0.163439,-0.006546,0.971524
holiday,0.029368,1.000000,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,0.043799,-0.020956,-0.005393,-0.000354,0.001731
workingday,-0.008126,-0.250491,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,-0.319111,0.119460,0.011594,0.002780,-0.003394
weather,0.008879,-0.007074,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.135918,-0.109340,-0.128655,-0.022740,0.012144
temp,0.258689,0.000295,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.467097,0.318571,0.394454,0.145430,0.257589
atemp,0.264744,-0.005215,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.462067,0.314635,0.389784,0.140343,0.264173
humidity,0.190610,0.001929,-0.010880,0.406244,-0.064949,-0.043536,1.000000,-0.318607,-0.348187,-0.265458,-0.317371,-0.278011,0.204537
windspeed,-0.147121,0.008409,0.013373,0.007261,-0.017852,-0.057473,-0.318607,1.000000,0.092276,0.091052,0.101369,0.146631,-0.150192
casual,0.096758,0.043799,-0.319111,-0.135918,0.467097,0.462067,-0.348187,0.092276,1.000000,0.497250,0.690414,0.302045,0.092722
registered,0.164011,-0.020956,0.119460,-0.109340,0.318571,0.314635,-0.265458,0.091052,0.497250,1.000000,0.970948,0.380540,0.169451


From this, we can see that casual and registered needs to be dropped due to high correlation. Besides, the month variable we created is also high correlated with seaon, so can be dropped as well

In [120]:
train_input = train_input.drop(['month', 'casual', 'registered', 'atemp'], axis=1)
train_input

,season,holiday,workingday,weather,temp,humidity,windspeed,count,hour
0,1,0,0,1,9.84,81,0.0000,16,0
1,1,0,0,1,9.02,80,0.0000,40,1
2,1,0,0,1,9.02,80,0.0000,32,2
3,1,0,0,1,9.84,75,0.0000,13,3
4,1,0,0,1,9.84,75,0.0000,1,4
...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,50,26.0027,336,19
10882,4,0,1,1,14.76,57,15.0013,241,20
10883,4,0,1,1,13.94,61,15.0013,168,21
10884,4,0,1,1,13.94,61,6.0032,129,22


Performing one hot encoding

In [121]:
train_input = pd.get_dummies(train_input, columns=['season','weather', 'holiday', 'workingday'])
train_input

,temp,humidity,windspeed,count,hour,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,holiday_0,holiday_1,workingday_0,workingday_1
0,9.84,81,0.0000,16,0,True,False,False,False,True,False,False,False,True,False,True,False
1,9.02,80,0.0000,40,1,True,False,False,False,True,False,False,False,True,False,True,False
2,9.02,80,0.0000,32,2,True,False,False,False,True,False,False,False,True,False,True,False
3,9.84,75,0.0000,13,3,True,False,False,False,True,False,False,False,True,False,True,False
4,9.84,75,0.0000,1,4,True,False,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,15.58,50,26.0027,336,19,False,False,False,True,True,False,False,False,True,False,False,True
10882,14.76,57,15.0013,241,20,False,False,False,True,True,False,False,False,True,False,False,True
10883,13.94,61,15.0013,168,21,False,False,False,True,True,False,False,False,True,False,False,True
10884,13.94,61,6.0032,129,22,False,False,False,True,True,False,False,False,True,False,False,True


Filling empty places

In [122]:
for column in train_input.columns:
    train_input.fillna(value=train_input[column].median())

train_input

,temp,humidity,windspeed,count,hour,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,holiday_0,holiday_1,workingday_0,workingday_1
0,9.84,81,0.0000,16,0,True,False,False,False,True,False,False,False,True,False,True,False
1,9.02,80,0.0000,40,1,True,False,False,False,True,False,False,False,True,False,True,False
2,9.02,80,0.0000,32,2,True,False,False,False,True,False,False,False,True,False,True,False
3,9.84,75,0.0000,13,3,True,False,False,False,True,False,False,False,True,False,True,False
4,9.84,75,0.0000,1,4,True,False,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,15.58,50,26.0027,336,19,False,False,False,True,True,False,False,False,True,False,False,True
10882,14.76,57,15.0013,241,20,False,False,False,True,True,False,False,False,True,False,False,True
10883,13.94,61,15.0013,168,21,False,False,False,True,True,False,False,False,True,False,False,True
10884,13.94,61,6.0032,129,22,False,False,False,True,True,False,False,False,True,False,False,True


Scaling features

In [123]:
from sklearn.preprocessing import StandardScaler

columns_to_scale = ['temp', 'humidity', 'windspeed', 'hour']
scaler = StandardScaler()
train_input[columns_to_scale] = scaler.fit_transform(train_input[columns_to_scale])

train_input

,temp,humidity,windspeed,count,hour,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,holiday_0,holiday_1,workingday_0,workingday_1
0,-1.333661,0.993213,-1.567754,16,-1.668944,True,False,False,False,True,False,False,False,True,False,True,False
1,-1.438907,0.941249,-1.567754,40,-1.524341,True,False,False,False,True,False,False,False,True,False,True,False
2,-1.438907,0.941249,-1.567754,32,-1.379739,True,False,False,False,True,False,False,False,True,False,True,False
3,-1.333661,0.681430,-1.567754,13,-1.235137,True,False,False,False,True,False,False,False,True,False,True,False
4,-1.333661,0.681430,-1.567754,1,-1.090534,True,False,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,-0.596935,-0.617666,1.617227,336,1.078500,False,False,False,True,True,False,False,False,True,False,False,True
10882,-0.702182,-0.253919,0.269704,241,1.223102,False,False,False,True,True,False,False,False,True,False,False,True
10883,-0.807428,-0.046064,0.269704,168,1.367704,False,False,False,True,True,False,False,False,True,False,False,True
10884,-0.807428,-0.046064,-0.832442,129,1.512307,False,False,False,True,True,False,False,False,True,False,False,True


Splitting into train and test data

In [124]:
from sklearn.model_selection import train_test_split

train_input, test_input = train_test_split(train_input, test_size = 0.20)

In [125]:
train_target = train_input['count']
train_input = train_input.drop('count', axis=1)

test_target = test_input['count']
test_input = test_input.drop('count', axis=1)

Testing out several models

XGBoost regressor

In [126]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xgb_regressor = xgb.XGBRegressor()
xgb_regressor.fit(train_input, train_target)
xgb_prediction = xgb_regressor.predict(test_input)

mse = mean_squared_error(xgb_prediction, test_target)
print("Root Mean Squared Error:", mse)

Root Mean Squared Error: 4823.867373468717


Decision tree regressor

In [127]:
from sklearn.tree import DecisionTreeRegressor

dt_regressor = DecisionTreeRegressor()

dt_regressor.fit(train_input, train_target)

dt_prediction = dt_regressor.predict(test_input)

mse = mean_squared_error(test_target, dt_prediction)
print("Mean Squared Error:", mse)

Mean Squared Error: 10424.877754820936


Random Forest regressor

In [128]:
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor()

rf_regressor.fit(train_input, train_target)

rf_prediction = rf_regressor.predict(test_input)

mse = mean_squared_error(rf_prediction, test_target)
print("Mean Squared Error:", mse)

Mean Squared Error: 5393.694687799297
